In [10]:
def main_part_vehicle(main_part):

    import re
    vehicles = ["機車","小客車","貨車", "大型車","電動自行車"] # 正規的交通工具
    verbs = ["騎乘","駕駛"] # 綫索1
    str_cues = ["車牌號碼"] # 綫索2
    vehicle = []
    target = []
    
    vehicles_final = ["機車","小客車","貨車", "大型車","電動自行車","駕車"] # 綫索都用完了還是沒捉到交通工具時的最後通牒
    
    main_part = main_part.strip().replace("\n","").replace(" ","") # 對文檔做處理

    
    for i in verbs: # 先抓綫索1
        iter = re.finditer(i, main_part)
        indices = [m.start(0) for m in iter]
        if indices:
            for j in indices:
                if "車牌號碼" in main_part[j:j+20]: # 如果 綫索1 綫索2 同時出現
                    for k in vehicles:
                        if k in main_part[j:j+25]:
                            vehicle.append(k)
                    if vehicle:
                        #print("Strong Cue!") # 抓對的機率很高，這個部分只是爲了檢驗才把字print出來，可刪去
                        return vehicle[0] # 既然綫索這麽強就直接抓了，以下就省略
                        
                else: # 如果只有綫索1 
                    for k in vehicles:
                        if k in main_part[j:j+25]:
                            vehicle.append(k)
                    if vehicle:
                        #print("Rather strong cue.") # 抓對的機率也很高，也抓了吧！
                        return vehicle[0]
    
    if not vehicle: # 如果綫索都沒有找到，只能出最後通牒了，只要看到vehicle_final都抓起來
        for i in vehicles_final:
            iter_ = re.finditer(i, main_part)
            indices = [m.start(0) for m in iter_]
            if indices:
                for j in indices:
                    target.append(j) # 把所有抓到的交通工具在文章中的位置記錄下來
        try:
            target = min(target) # 這個判斷是因爲發現在文中常常會先説被告的罪行，
            # 才説事情經過，所以在文中最早出現的交通工具比較有可能是被告駕駛的交通工具，
            # 這是我自己看到的經驗法則，不排除這個部分會抓錯，至少我自己抽查了蠻常都是對的。
            vhc_clue = main_part[target]
            if vhc_clue == "機":
                vehicle = "機車"
            elif vhc_clue == "小":
                vehicle = "小客車"
            elif vhc_clue == "駕": # 很多的文章（尤其桃園（可到30多%），士林，臺中，臺東，花蓮）
            # 都沒有明確表明交通工具，都寫"駕車"，個人判斷較高機率為小客車，一樣有判斷錯誤的可能
                vehicle = "小客車"
            elif vhc_clue == "貨" or vhc_clue == "大":
                vehicle = "大型車"
            elif vhc_clue == "電":
                vehicle = "電動自行車"
            #print("Not so sure......") # 一樣這個只是方便自己做檢驗，可刪去    
            return vehicle
        except:
            return "Error"

# 這樣的結果是有729篇文章會error。當然有抓到結果的當中可能會有錯誤的部分不過 應 該 不 會 多 過 對 的       


In [2]:
import glob
files = glob.glob(r'D:\Google雲端硬碟\NCKU\法經\法經報告\procedure\判決書\*\*.txt')
len(files)

26885

In [14]:
filenames = []
vehicles = []
for i,file in enumerate( files ):
    if i % 2700 == 0:
        print(i)
    with open(file,'r',encoding='utf-8-sig') as f:
        txt = f.read()
    vehi = main_part_vehicle(txt)
    filenames.append(file)
    vehicles.append(vehi)

0
2700
5400
8100
10800
13500
16200
18900
21600
24300


In [19]:
set(vehicles)

{'Error', '大型車', '小客車', '機車', '貨車', '電動自行車'}

In [25]:
new= vehicles
for i,t in enumerate(vehicles):
    if t == 'Error':
        new[i] = ''

{'', '大型車', '小客車', '機車', '貨車', '電動自行車'}

In [39]:
tran = {'大型車':'v_Lcar','小客車':'v_car','機車':'v_motor','貨車':'v_truck','電動自行車':'v_elebike','':'' }
for i,t in enumerate(vehicles):
    if t in tran:
        vehicles[i] = tran[t]
set(vehicles)

{'', 'v_Lcar', 'v_car', 'v_elebike', 'v_motor', 'v_truck'}

In [42]:
a= filenames

In [43]:
a[0]

'D:\\Google雲端硬碟\\NCKU\\法經\\法經報告\\procedure\\判決書\\南投\\南投_2019-01-02_投交簡_576.txt'

In [44]:
for i,t in enumerate(a):
    a[i] = t.split('\\')[-1]
a[10]

'南投_2019-01-11_埔交簡_181.txt'

In [45]:
filenames = a

In [46]:
import pandas as pd
df = pd.DataFrame(data={'index':filenames,'vehicle': vehicles})
path = 'D:/Google雲端硬碟/NCKU/法經/法經報告/procedure/csv/index_vehicle.csv'
df.to_csv(path,encoding='utf-8-sig',index=False)
data = pd.read_csv('D:/Google雲端硬碟/NCKU/法經/法經報告/procedure/csv/data26885_5.csv',encoding='utf-8-sig')
data.shape

(26885, 129)

In [48]:
df.set_index(['index'],inplace = True)
data.set_index(['index'],inplace = True)
data2 = pd.concat([data,df],axis='columns')
data2.reset_index(inplace=True)
data2

,index,court,date,case_kind,case_num,judge,jail,week,laws,'crime_1',...,shilin,taichung,tainan,taipei,taitung,taoyuan,yilan,yunlin,alcohol,vehicle
0,南投_2019-01-02_投交簡_576.txt,nantou,2019/1/2,投交簡,576,孫于淦,2,1,"'suit_449', 'suit_454', 'crime_185-3', 'crime_...",0,...,0,0,0,0,0,0,0,0,1.075,v_motor
1,南投_2019-01-07_埔交簡_2.txt,nantou,2019/1/7,埔交簡,2,孫于淦,3,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_41'",0,...,0,0,0,0,0,0,0,0,0.540,v_truck
2,南投_2019-01-07_投交簡_521.txt,nantou,2019/1/7,投交簡,521,楊國煜,2,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_41'",0,...,0,0,0,0,0,0,0,0,0.290,v_car
3,南投_2019-01-07_投交簡_524.txt,nantou,2019/1/7,投交簡,524,孫于淦,5,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_41'",0,...,0,0,0,0,0,0,0,0,0.250,v_motor
4,南投_2019-01-07_投交簡_531.txt,nantou,2019/1/7,投交簡,531,黃俊明,5,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_...",0,...,0,0,0,0,0,0,0,0,0.430,v_motor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26880,高雄_2019-06-18_審交易_437.txt,kaohsiung,2019/6/18,審交易,437,詹尚晃,8,25,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,...,0,0,0,0,0,0,0,0,0.700,v_motor
26881,高雄_2019-07-16_審交易_528.txt,kaohsiung,2019/7/16,審交易,528,詹尚晃,16,29,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,...,0,0,0,0,0,0,0,0,0.660,v_motor
26882,高雄_2019-08-15_審交易_625.txt,kaohsiung,2019/8/15,審交易,625,詹尚晃,10,33,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,...,0,0,0,0,0,0,0,0,0.500,v_motor
26883,高雄_2019-08-20_審交易_652.txt,kaohsiung,2019/8/20,審交易,652,詹尚晃,5,34,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,...,0,0,0,0,0,0,0,0,0.590,v_motor


In [49]:
data2.to_csv('D:/Google雲端硬碟/NCKU/法經/法經報告/procedure/csv/data26885_6.csv',encoding='utf-8-sig',index=False)
data2.to_csv('D:/Google雲端硬碟/NCKU/法經/備份檔案/data26885_6.csv',encoding='utf-8-sig',index=False)